In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
!pip install gensim --upgrade
!pip install tensorflow_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_text-2.9.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 1.6 MB 51.4 MB/s 
     |████████████████████████████████| 438 kB 54.6 MB/s 
     |████████████████████████████████| 5.8 MB 29.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninst

In [ ]:
import nltk
import pandas as pd
import numpy as np
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df = pd.read_csv("/content/imdb.csv")

In [ ]:
df = df.iloc[:30000]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [ ]:
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(17736013, 18588780)

In [ ]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 29854/29854 [35:48<00:00, 13.89it/s]


In [ ]:
X = np.array(X)

In [ ]:
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
model = Sequential([
  layers.Dense(32, activation="softmax"),
  layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
)

In [ ]:
model.fit(
  x = X_train,
  y = y_train,
  epochs = 10,
  verbose =1
)

Epoch 1/100
747/747 [==============================] - 2s 2ms/step - loss: 0.6872 - accuracy: 0.6067
Epoch 2/100
747/747 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.8205
Epoch 3/100
747/747 [==============================] - 1s 2ms/step - loss: 0.4378 - accuracy: 0.8281
Epoch 4/100
747/747 [==============================] - 1s 2ms/step - loss: 0.4155 - accuracy: 0.8321
Epoch 5/100
747/747 [==============================] - 1s 2ms/step - loss: 0.3984 - accuracy: 0.8331
Epoch 6/100
747/747 [==============================] - 1s 2ms/step - loss: 0.3893 - accuracy: 0.8335
Epoch 7/100
747/747 [==============================] - 1s 2ms/step - loss: 0.3844 - accuracy: 0.8333
Epoch 8/100
747/747 [==============================] - 2s 2ms/step - loss: 0.3808 - accuracy: 0.8334
Epoch 9/100
747/747 [==============================] - 2s 2ms/step - loss: 0.3772 - accuracy: 0.8370
Epoch 10/100
747/747 [==============================] - 1s 2ms/step - loss: 0.3745 - accura

In [ ]:
model.evaluate(X_test, y_test)